In [10]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [11]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")


[API KEY]
sk-proj-1kwX9MIvUjxZZ1qK6cgIT3BlbkFJXgYQCz0QqlvIp54pDw0E


cmd로 gguf파일 다운로드하기

In [12]:
%%bash
huggingface-cli download \
  heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF \
  ggml-model-Q5_K_M.gguf \
  --local-dir C:/Users/dm705/Study \
  --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/heegyu/EEVE-Korean-Instruct-10.8B-v1.0-GGUF/resolve/main/ggml-model-Q5_K_M.gguf to C:\Users\dm705\.cache\huggingface\hub\tmpojk2nbhj


## 1. 모델 불러오기(EEVE-Korean-10.8B)

In [1]:
"""
requirements: pip install langchain
"""

from langchain_community.chat_models import ChatOllama   
from langchain_core.output_parsers import StrOutputParser # 출력을 Str
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager
from langchain import LLMChain

llm = ChatOllama(
    model="EEVE-Korean-10.8B:latest",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

## 2. 프롬프트 템플릿 설정

In [2]:
import torch
if torch.cuda.is_available():
    torch.cuda.set_device(torch.device("cuda:0"))
    torch.cuda.empty_cache()

In [3]:
from langchain import LLMChain

# 2. 프롬프트 템플릿 만들기
template = """
입력된 텍스트를 문장 단위로 나누어 주세요:
나뉜 문장을 파이썬 리스트 형식으로 답변해주세요.
- [출력예시]:
>>> split_sentences: ['문장1 내용', '문장2 내용', '문장3 내용', ...]
{text}
"""
prompt = ChatPromptTemplate.from_template(template)

# 3. 출력 파서 설정하기
output_parser = StrOutputParser()

# 4. LLMChain 만들기
llm_chain = LLMChain(llm=llm, prompt=prompt, output_parser=output_parser)
# chain = llm | prompt | output_parser

# 5. 텍스트 입력하고 실행하기
input_text = """
장애인 업무의 한계를 없애다 상시 근로자 100,016 명 / 장애인 근로자 1,386 명 (1.54%) * 2018.12 월 기준 장애물 없는 생활환경 (Barrier Free) 자체 인증 제도를 각 사업장에 도입 상담심리사를 상주시켜 직무갈등 , 대인관계 등 고충을 해결 업무에 집중할 환경을 마련 장애인 고객의 입장에서 바라본 기술개발에도 적극 참여 장애인은 단순 업무만 해야 하나 ? ‘ 삼성전자 맞춤형 인재 ’ 로 성장하는데 있어서 비장애인과 장애인의 구분은 의미가 없다고 판단 , 직무교육을 거쳐 고용된 장애인은 각 사업장에서 전기 · 전자 , 정보기술 , 기계디자인 분야에서 근무 중 장애인고용 기업 사례 장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련
"""

# LLMChain 실행하기
result = llm_chain.invoke({"text": input_text})
# result = chain.invoke({"text": input_text})
# print(result)

c:\Users\dm705\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


['삼성전자의 장애인 업무 한계 없앤다', '2018년 12월 기준 상시 근로자 100,016명 / 장애인 근로자 1,386명 (1.54%)', '장애물 없는 생활환경(Barrier Free) 자체 인증 제도 도입', '상담심리사 배치로 직무갈등 및 대인관계 고충 해결', '업무 집중 환경을 위한 지원', '장애인의 기술개발 참여', '직무교육을 거쳐 고용된 장애인 각 사업장 근무', '전기·전자, 정보기술, 기계디자인 분야 종사', '장애인고용 기업 사례', '장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련']

## 3. 전처리 - 문장 구분하기 (using Ollama)

In [10]:
import pandas as pd

# 5. CSV 파일 읽고 처리하기
input_csv_path = "extract_text_[PPT] 직장 내 장애인 인식개선 교육 표준교안 PPT(국문)_수정.csv"
df = pd.read_csv(input_csv_path)

# 텍스트 컬럼 이름 지정
text_column = "Text"

# 결과 저장할 리스트
results = []

# 각 텍스트에 대해 처리
for text in df[text_column]:
    result = llm_chain.run({"text": text})
    results.append(result)

# 결과를 데이터프레임에 추가
df["processed_text"] = results

# 결과를 새로운 CSV 파일로 저장
output_csv_path = "output.csv"
df.to_csv(output_csv_path, index=False)

장애인 업무의 한계를 없애다\n상시 근로자 100,016 명 / 장애인 근로자 1,386 명 (1.54%) * 2018.12 월 기준\ndeptbarrier Free 생활 환경 자체 인증 제도를 각 사업장마다 도입\ndept상담심리사를 상주시켜 직무갈등, 대인관계 등 고충을 해결하여 업무에 집중할 환경을 마련\ndept장애인은 단순 업무만 해야 하나요? \ndept편견을 걷어내면 인재가 보입니다 직장 내 장애인 인식개선 교육 장애인 고용제공하신 입력 텍스트를 문장 단위로 나누어

KeyboardInterrupt: 

In [ ]:
df.head()

,Slide Number,Text,processed_text
0,1,편견 을 걷어내면 인재 가 보입니다 직장 내 장애인 인식개선 교육 장애인 고용,1. 편견 걷어내기\n2. 인재가 보인다\n3. 직장 내 장애 인식 개선 교육\n4...
1,2,"장애인 고용 , 기업에게 어떤 의미가 있을까 함께 일하는 동료 이해하기 장애인 ...","입력된 텍스트를 문장 단위로 나눈 결과입니다:\r\n\r\n* 장애인 고용, 기업에..."
2,3,"01 장애인 고용 , 기업에게 어떤 의미가 있을까",1. 장애인 고용이 기업에 미치는 영향\n2. 직원 다양성의 중요성\n3. 장애를 ...
3,4,차별도 우대도 없었던 게임 산업 분위기 속 장애인 고용의 돌파구 2011 년 ...,다음은 입력된 텍스트를 문장으로 나눈 버전입니다:\r\n\r\n* 차별도 우대도 없...
4,5,"장애인 업무의 한계를 없애다 상시 근로자 100,016 명 / 장애인 ...","문장 단위 분할:\n1. 장애인 업무의 한계를 없애다\n2. 상시 근로자 100,0..."


In [ ]:
df.iloc[4]['processed_text']

'문장 단위 분할:\\n1. 장애인 업무의 한계를 없애다\\n2. 상시 근로자 100,016 명 / 장애인 근로자 1,386 명 (1.54%) * 2018년 12월 기준\\n3. 장애물 없는 생활환경 (Barrier Free) 자체 인증 제도를 각 사업장 도입\\n4. 상담심리사를 상주시켜 직무갈등, 대인관계 등 고충 해결\\n5. 업무에 집중할 환경을 마련\\n6. 장애인 고객의 입장에서 바라본 기술개발에도 적극 참여\\n7. 장애인이라고 해서 단순한 업무만 해야 하나요? ‘삼성전자 맞춤형 인재’로 성장하는 데에 있어서 비장애인과 장애인 구분은 의미가 없다고 판단\\n8. 직무교육을 거쳐 고용된 장애인, 각 사업장에서 전기·전자, 정보기술, 기계디자인 분야에서 근무 중\\n9. 장애인고용 기업 사례: 장애인 고용 전문가와 함께 다양한 장애인 고용 대책 마련'